In [1]:
from typing import Any

import lightning.pytorch as pl
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow
from torch.utils.data import DataLoader, Dataset

from src.ml.data.splitting import create_data_splits
from src.ml.modeling import (
    model_factory,
    optimizer_factory,
)
from src.ml.utils.set_seed import set_seed

from datetime import datetime
from loguru import logger
from pathlib import Path
from shutil import copy, copytree, rmtree

import yaml

from src.ml.train_neural_network import train_neural_network
from src.ml.data import data_sets_factory
from src.ml.preprocessing import preprocessing_factory
from src.ml.utils.set_seed import set_seed

In [2]:
CONFIG_FILE = Path("src/ml/config.yaml")

In [3]:
torch.set_default_device(torch.device("cpu"))

In [4]:
# load config file

logger.info("Loading config file.")

with open(CONFIG_FILE, "r") as f:
    config = yaml.safe_load(f)

2024-12-12 22:13:12.721 | INFO     | __main__:<module>:3 - Loading config file.


In [5]:
# load data

logger.info("Loading data.")

data_sets = data_sets_factory(**config["data_set"])
logger.info("Loaded {} data sets.", len(data_sets))

# preprocess data

logger.info("Start preprocessing.")

for preprocessing_step in config["preprocessing"]:
    logger.info("Perform {} preprocessing.", preprocessing_step["name"])

    transform = preprocessing_factory(**preprocessing_step)
    data_sets = [transform(data_set) for data_set in data_sets]

dataset = data_sets[0]

2024-12-12 22:13:13.420 | INFO     | __main__:<module>:3 - Loading data.
2024-12-12 22:13:22.207 | INFO     | __main__:<module>:6 - Loaded 1 data sets.
2024-12-12 22:13:22.207 | INFO     | __main__:<module>:10 - Start preprocessing.
2024-12-12 22:13:22.207 | INFO     | __main__:<module>:13 - Perform add_taxa_names preprocessing.
2024-12-12 22:13:23.160 | INFO     | __main__:<module>:13 - Perform add_clade_information preprocessing.
2024-12-12 22:13:25.609 | INFO     | __main__:<module>:13 - Perform remove_tree preprocessing.


In [6]:
train_dataset, val_dataset, test_dataset = create_data_splits(
    dataset, **config["training"]["splitting_config"]
)

In [7]:
train_loader = DataLoader(train_dataset, **config["training"]["dataloader_config"])
test_loader = DataLoader(test_dataset)
val_loader = DataLoader(val_dataset)

optimizer = optimizer_factory(**config["training"]["optimizer_config"])
model = model_factory(
    optimizer=optimizer,
    dim=len(train_dataset[0]["branch_lengths"]),
    context_dim=len(train_dataset[0]["clades_one_hot"]),
    **config["training"]["model_config"],
)

In [8]:
from src.ml.modeling.conditional_tree_flow import ConditionalTreeFlow

model = ConditionalTreeFlow.load_from_checkpoint("ml_data/models/debug_setup/epoch=88-val_loss=-15.08-v1.ckpt")

In [13]:
sample = next(iter(test_loader))
latent = model.forward(sample)
output = model.inverse(latent)
sample, output

({'trees_file': ['data/mcmc_runs/yule-10_140.trees'],
  'tree_index': tensor([34217]),
  'taxa_names': [('0',),
   ('8',),
   ('6',),
   ('5',),
   ('7',),
   ('3',),
   ('4',),
   ('9',),
   ('1',),
   ('2',)],
  'clades': [tensor([3]),
   tensor([12]),
   tensor([19]),
   tensor([31]),
   tensor([96]),
   tensor([224]),
   tensor([255]),
   tensor([768]),
   tensor([1023])],
  'branch_lengths': tensor([[0.0105, 0.0028, 0.0049, 0.0012, 0.0004, 0.0042, 0.0075, 0.0053, 0.0362]]),
  'clades_one_hot': tensor([[1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 1.]])},
 {'branch_lengths': tensor([[0.0105, 0.0028, 0.0049, 0.0012, 0.0004, 0.0042, 0.0075, 0.0053, 0.0362]],
         grad_fn=<AddBackward0>),
  'clades_one_hot': tensor([[1., 1., 0., 0., 0., 0., 0., 0., 

In [23]:
latent = model.forward(next(iter(train_loader)))

In [28]:
import numpy as np

z = latent["z"].detach().cpu().numpy()

In [50]:
import seaborn as sns

latent = next(iter(test_loader))
latent["branch_lengths"] = torch.distributions.normal.Normal(loc=0.0, scale=1.0).sample(latent["branch_lengths"].shape) 
output = model.inverse(latent)

output["z"][0], next(iter(test_loader))["branch_lengths"][0]

(tensor([-0.8882,  0.4503,  0.9087, -0.8671, -0.5895, -0.5164,  1.9490, -2.5399,
         -0.1192], grad_fn=<SelectBackward0>),
 tensor([0.0164, 0.0040, 0.0034, 0.0041, 0.0023, 0.0007, 0.0032, 0.0058, 0.0421]))